In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"
# os.environ['CUDA_LAUNCH_BLOCKING'] = "5"

In [2]:
pip install lime

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install safetensors


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [2]:
import lime
import torch
import pandas as pd
from ast import literal_eval
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from lime.lime_text import LimeTextExplainer
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification , BertTokenizer , BertForSequenceClassification

2024-09-30 02:25:55.433824: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-30 02:25:55.433948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-30 02:25:55.435421: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-30 02:25:55.444230: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 02:25:56.540501: W tensorflow/compiler/tf2

In [3]:
class BertWithAttentionSupervision(nn.Module):
    def __init__(self, num_labels=2):
        super(BertWithAttentionSupervision, self).__init__()
        self.bert_classifier = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=num_labels)
        self.attention_loss_fn = nn.BCEWithLogitsLoss()

    def forward(self, input_ids, attention_mask, labels=None, ground_truth_attention=None):
        outputs = self.bert_classifier(input_ids=input_ids,
                                       attention_mask=attention_mask,
                                       #token_type_ids=token_type_ids,
                                       labels=labels,
                                       output_attentions=True,
                                       return_dict=True)

        classification_loss = outputs.loss
        logits = outputs.logits

        # Extract attention scores from the final layer
        attention_scores = outputs.attentions[-1][:, :, 0, :]  # CLS token attention across all heads
        avg_attention_scores = attention_scores.mean(dim=1)  # Average attention scores across heads

        # Calculate attention loss only if ground_truth_attention is provided
        attention_loss = 0
        if ground_truth_attention is not None:
            if ground_truth_attention.size(0) != avg_attention_scores.size(0):
                raise ValueError(f"Expected ground_truth_attention to be of size {avg_attention_scores.size(0)}, but got {ground_truth_attention.size(0)}")
            attention_loss = self.attention_loss_fn(avg_attention_scores, ground_truth_attention)

        return logits

In [4]:
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model = BertWithAttentionSupervision(num_labels=2).to(device)

# Load the model state dict with map_location for CPU
loaded_model.load_state_dict(torch.load("/home/nagendra/system_files/bin/english/with_rationales/xlmrl/xlmr_attention_model1.pth", map_location=device))

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_4075755/4124861956.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explic

<All keys matched successfully>

In [6]:
# Define class names
class_names = ['non-hate', 'hate']

# Predictor function for LIME
def predictor(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        logits = loaded_model(**inputs)
    probas = F.softmax(logits, dim=1).detach().cpu().numpy()
    return probas

# Initialize LIME Text Explainer
explainer = LimeTextExplainer(class_names=class_names, split_expression='\s+')

data = pd.read_csv('/home/nagendra/system_files/bin/english/with_rationales/english_clean_new.csv')

texts = data['cleaned'].astype(str).tolist()
labels = data['is_hateful'].tolist()

train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.15, random_state=42  # stratify to maintain label distribution
)

In [ ]:
data_for_excel = []

# Loop through each text in test_texts
for i, text in enumerate(test_texts):
    # Explain the instance for the current text
    exp = explainer.explain_instance(text, predictor, num_features=200, num_samples=500)

    # Get the explanation map and list
    explanation_map = exp.as_map()
    explanation_list = exp.as_list()

    # Concatenate the lists based on matching y values
    combined_explanation_list = [
        (x, y, z) for (x, y) in explanation_list for (z, y2) in explanation_map[1] if y == y2
    ]

    # Sort the combined list based on the third element (z)
    sorted_combined_explanation_list = sorted(combined_explanation_list, key=lambda item: item[2])

    # Extract tokens and exact_hateful values
    tokens = [item[0] for item in sorted_combined_explanation_list]
    exact_hateful = [item[1] for item in sorted_combined_explanation_list]
    is_hateful = [1 if item[1] > 0 else 0 for item in sorted_combined_explanation_list]

    # Add the data to the list for Excel export
    data_for_excel.append({
        'full_text': text,    # Full text of the current instance
        'tokens': tokens,     # List of tokens
        'is_hateful': is_hateful,  # List of is_hateful values
        'exact_hateful': exact_hateful
    })

# Convert the list of dictionaries to a DataFrame
df_xcel = pd.DataFrame(data_for_excel)

# Save the DataFrame to an Excel file
df_xcel.to_excel("english_xlmr_rational.xlsx", index=False)